In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("restaurant-review-average")
sc = SparkContext(conf=conf)

In [2]:
!pwd

/home/tutor/SparkCourse


In [3]:
filepath = "/home/tutor/SparkCourse/data/restaurant_reviews.csv"

In [4]:
lines = sc.textFile(f"file:///{filepath}")
lines.collect()

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [5]:
header = lines.first()
rows = lines.filter(lambda row : row != header)
rows.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [6]:
# 각 행을 파싱하기 위한 함수
def parse(row):
    fields = row.split(",")
    
    # 카테고리 항목 얻어오기
    category = fields[2]
    
    # 리뷰 개수 얻어오기 - 정수 형태로 파싱
    reviews = fields[3]
    reviews = int(reviews)
    
    return category, reviews

In [7]:
parse('7,쌀국수,아시안,312,')

('아시안', 312)

In [8]:
# RDD내의 모든 row에 대해 parse 함수를 적용하고 추출(map)
category_reviews = rows.map(parse)
category_reviews

PythonRDD[4] at RDD at PythonRDD.scala:53

In [9]:
# 확인
category_reviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

카테고리 별 리뷰 평균
* 각 카테고리 별 리뷰 개수
* 카테고리 종류의 개수
    * 중식은 몇건, 분식은 몇건...

In [13]:
category_reviews_count = category_reviews.mapValues(lambda reviews : (reviews, 1))
category_reviews_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [14]:
reduce = category_reviews_count.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))
reduce.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

예를 들어 **중식**의 `x`, `y`

- 첫 번째 중식 : `(125, 1)` -> `x`
- 두 번째 중식 : `(235, 1)` -> `y`

------
- 리뷰의 개수끼리 합치기 : `x[0] + y[0]`
- 건수 끼리 합치기 : `x[1] + y[1]`



In [15]:
average = reduce.mapValues(lambda x : x[0] / x[1] )
average.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

```SQL
SELECT AVG(reviews)
FROM category_reviews
GROUP BY CATEGORY
```

In [16]:
sc.stop()